In [1]:
import requests
import bz2
import json

In [101]:
#http://docs.python-requests.org/en/master/user/quickstart/
#https://github.com/liampauling/betfair/blob/master/betfairlightweight/endpoints/historic.py

In [100]:
teams = ['ADO Den Haag', 'Ajax', 'AZ Alkmaar', 'De Graafschap', 'Emmen', 'Excelsior', 'Feyenoord', 'Fortuna Sittard', 
         'Groningen', 'Heerenveen','Heracles', 'Breda', 'Zwolle', 'PSV', 'Utrecht', 'Vitesse', 'VVV','Willem II']

In [12]:
url = 'http://historicdata.betfair.com/api/DownloadListOfFiles'
headers = {'ssoid': 'TOKEN'}

data = {"sport":"soccer",
    "plan":"Basic Plan",
    "fromDay" : 1,
    "fromMonth": 9,
    "fromYear" : 2018,
    "toDay": 30,
    "toMonth" : 9,
    "toYear": 2018,
    "eventId": None,
    "eventName": None,
    "marketTypesCollection": ["MATCH_ODDS"],
    "countriesCollection" : ["NL"],
    "fileTypeCollection" : ["M"]}

In [19]:
# download the filenames
response = requests.request("POST", url, headers=headers,data = data)

In [38]:
file_names = response.text[1:-1].split(',')

In [42]:
print(file_names[0])
print(file_names[1])
print("There are {} files".format(len(file_names)))

"/data/xds/historic/BASIC/28880276/1.147452117.bz2"
"/data/xds/historic/BASIC/28868359/1.147085739.bz2"
There are 152 files


In [93]:
# now download the actual data
file_name = file_names[0][1:-1]

for i in file_names:
    
    file_name = i[1:-1]
    
    r = requests.get("http://historicdata.betfair.com/api/DownloadFile",
                params={'filePath': file_name},
                headers=headers,
                stream=True)

    local_filename = 'data/' + file_name.split("/")[-1]

    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [96]:
for i in file_names:
    #print(i)
    file_name = i[1:-1]
    local_filename = 'data/' + file_name.split("/")[-1]
    
    bz_file = bz2.BZ2File(local_filename)
    line_list = bz_file.readlines()
    
    # only interested in the first line for now
    a = line_list[0].decode('ASCII').strip()
    d = json.loads(a)
    print(d['mc'][0]['marketDefinition']['eventName'])

Vitesse Arnhem U19 v Sparta Rotterdam U19
VVV v Heerenveen
PSV v Willem II
Excelsior v ADO Den Haag
Fortuna Sittard v Utrecht
Groningen v Zwolle
FC Lienden v De Treffers
Heracles v AZ Alkmaar
Vitesse v Ajax
Feyenoord v Breda
RKC Waalwijk (Res) v Willem II Tilburg (Res)
Burton v Walsall
Oxford Utd v Fulham U21
Grimsby v Notts Co
Oldham v Barnsley
Southend v Cambridge Utd
Scunthorpe v Wolves U21
Portsmouth v Gillingham
Northampton v Wycombe
Luton v Brighton U21
Rochdale v Bury
Macclesfield v Blackpool
Carlisle v Morecambe
Forest Green v Cheltenham
Charlton v AFC Wimbledon
Colchester v Southampton U21
Tranmere v Crewe
Crawley Town v Tottenham U21
Sunderland v Stoke U21
MK Dons v Peterborough
Excelsior-Bdrect (W) v ADO Den Haag (W)
Twente (W) v PSV (W)
Ajax (W) v Zwolle (W)
VV Alkmaar (W) v Achilles 29 (W)
Dordrecht v Telstar
Cambuur v Waalwijk
Jong Utrecht v Jong PSV
Almere City v MVV Maastricht
Go Ahead Eagles v Volendam
Helmond Sport v Den Bosch
Sparta Rotterdam v FC Eindhoven
VVSB v HH